# FLAML demo

Autorzy: Sebastian David Botero Leonik, Bartosz Pokora, Franciszek Saliński

Agenda:
1. Czym jest FLAML?
2. Dane i baseline model
3. AutoML z FLAML
4. Porównanie wyników i wnioski

## Czym jest FLAML?

**FLAML** to open-source'owa biblioteka AutoML stworzona przez badaczy z Microsoft.

FLAML = *Fast and Lightweight AutoML Library* / *A Fast Library for Automated Machine Learning*

Źródła:
- [Dokumentacja FLAML](https://microsoft.github.io/FLAML/)
- [Repozytorium FLAML na GitHubie](https://github.com/microsoft/FLAML)
- [Publikacja FLAML](https://arxiv.org/pdf/1911.04706)

Główne cechy FLAML:
- Automatyzuje wybór modelu i optymalizację hiperparametrów
- Szybkość i lekkość, pozwala na ustalenie `time_budget`
- Używa API zgodnego z scikit-learn
- Obsługuje różne zadania ML: klasyfikacja, regresja, klasteryzacja, prognozowanie szeregów czasowych,...

In [ ]:
# Instalacja FLAML
#!pip install flaml[automl]

In [ ]:
# Importy
import warnings
warnings.filterwarnings("ignore")

import time

from flaml import AutoML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

RANDOM_STATE = 123
np.random.seed(RANDOM_STATE)

## Dane i baseline model

Wykorzystamy zbiór danych z **OpenML**: `bank-marketing`. Na początek zbudujemy prosty pipeline z modelem regresji logistycznej, ocenimy jego jakość i zmierzymy czas trenowania.

Dane: [bank-marketing dataset](https://www.openml.org/search?type=data&sort=runs&id=1461&status=active)

In [ ]:
X, y = fetch_openml(data_id=1461, as_frame=True, return_X_y=True)

# Zamiana etykiet na 0/1
y = y.astype(str)
y = (y == "2").astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

X_train.shape, X_test.shape

In [ ]:
X_train.head()

In [ ]:
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

print(f"Liczba cech: {X_train.shape[1]}")
print(f"Zmienne numeryczne: {num_cols}")
print(f"Zmienne kategoryczne: {cat_cols}")

In [ ]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop"
)

X_train_prep = preprocess.fit_transform(X_train)
X_test_prep  = preprocess.transform(X_test)

feature_names = preprocess.get_feature_names_out()
X_train_prep = pd.DataFrame(X_train_prep, columns=feature_names, index=X_train.index)
X_test_prep  = pd.DataFrame(X_test_prep,  columns=feature_names, index=X_test.index)

In [ ]:
log_reg = LogisticRegression(max_iter=1000, solver="liblinear", random_state=RANDOM_STATE)

param_grid = {
    "C": [0.1, 0.25, 0.5, 1, 2, 4, 10],
    "penalty": ["l1", "l2"],
}

grid_search = GridSearchCV(
    log_reg,
    param_grid=param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1
)

start_time = time.time()
grid_search.fit(X_train_prep, y_train)
end_time = time.time()

train_time_baseline = end_time - start_time

best_model = grid_search.best_estimator_
test_proba = best_model.predict_proba(X_test_prep)[:, 1]
test_pred = (test_proba >= 0.5).astype(int)

accuracy_baseline = accuracy_score(y_test, test_pred)
roc_auc_baseline = roc_auc_score(y_test, test_proba)

print("Najlepsze parametry:", grid_search.best_params_)
print(f"Training time: {train_time_baseline:.4f} seconds")
print(f"Accuracy: {accuracy_baseline:.4f}")
print(f"ROC AUC: {roc_auc_baseline:.4f}")

## AutoML z FLAML

Użyjemy teraz FLAML do automatycznego znalezienia najlepszego modelu i jego hiperparametrów dla tego samego zadania klasyfikacji.

Do przetworzenia danych użyjemy tego samego pipeline'u co wcześniej.

In [ ]:
automl = AutoML()

automl_settings = {
    # Czas (w sekundach), który chcemy przeznaczyć na uczenie
    "time_budget": 20,
    # Metryka do optymalizacji
    "metric": "roc_auc",
    # Rodzaj zadania
    "task": "classification",
    # Random seed
    "seed": RANDOM_STATE,
    # Lista modeli do sprawdzenia
    "estimator_list": ["rf", "extra_tree", "catboost", "lgbm"],
    # Plik do logów
    "log_file_name": "flaml_automl.log",
}

start_time = time.time()
automl.fit(X_train=X_train_prep, y_train=y_train, **automl_settings)
end_time = time.time()

train_time_flaml = end_time - start_time

print("Najlepszy estymator:", automl.best_estimator)
print("Najlepsza konfiguracja (best_config):")
print(automl.best_config)

In [ ]:
test_proba = automl.predict_proba(X_test_prep)[:, 1]
test_pred = (test_proba >= 0.5).astype(int)

accuracy_flaml = accuracy_score(y_test, test_pred)
roc_auc_flaml = roc_auc_score(y_test, test_proba)

print(f"Training time (FLAML): {train_time_flaml:.4f} seconds")
print(f"Accuracy (FLAML): {accuracy_flaml:.4f}")
print(f"ROC AUC (FLAML): {roc_auc_flaml:.4f}")

## Porównanie wyników i wnioski

Porównajmy najlepsze hiperparametry oraz wyniki baseline'u i modelu znalezionego przez FLAML pod względem ROC AUC oraz czasu treningu.

In [ ]:
print("=== Baseline ===")
print("Najlepszy estymator:", best_model)
print("Najlepsze parametry:", grid_search.best_params_)

print("\n=== FLAML ===")
print("Najlepszy estymator:", automl.best_estimator)
print("Najlepsze parametry:")
print(automl.best_config)

In [ ]:
results = pd.DataFrame({
    "Model": ["LogReg + GSCV", "FLAML AutoML"],
    "ROC AUC (test)": [roc_auc_baseline, roc_auc_flaml],
    "Accuracy (test)": [accuracy_baseline, accuracy_flaml],
    "Czas treningu [s]": [train_time_baseline, train_time_flaml],
})

results["ROC AUC (test)"] = results["ROC AUC (test)"].round(4)
results["Accuracy (test)"] = results["Accuracy (test)"].round(4)
results["Czas treningu [s]"] = results["Czas treningu [s]"].round(3)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 5))

sns.barplot(
    data=results,
    x="Model",
    y="ROC AUC (test)",
    ax=ax[0]
)
ax[0].set_title("Porównanie jakości modeli")
ax[0].set_xlabel("")
ax[0].set_ylabel("ROC AUC (test)")
ax[0].bar_label(ax[0].containers[0], fmt="%.2f")

sns.barplot(
    data=results,
    x="Model",
    y="Czas treningu [s]",
    ax=ax[1]
)
ax[1].set_title("Porównanie czasu uczenia")
ax[1].set_xlabel("")
ax[1].set_ylabel("Czas treningu [s]")
ax[1].bar_label(ax[1].containers[0], fmt="%.2f")

plt.tight_layout()
plt.show()

### Podsumowanie
- FLAML pozwala na automatyzację wyboru modelu i optymalizacji hiperparametrów w szybki i efektywny sposób
- Możemy łatwo zaoszczędzić czas, ilość kodu i zasoby obliczeniowe
- Na naszym przykładzie FLAML w krótszym czasie znalazł odrobinę lepszy model niż ręcznie stworzony pipeline z regresją logistyczną i GridSearchCV


### Dziękujemy za uwagę :)